In [ ]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd
import sklearn

In [ ]:
# 模拟数据
X = np.random.randn(100) * 5 +7
error = np.random.randn(100) * 50
Y_obs = 5 * X ** 2 + 10 * X + error

plt.scatter(X, Y_obs, s=5)

In [ ]:
# Ridge regularization parameter (lambda for ridge penalty)
lambda_ridge = 0.1  # ridge regression 的 lambda, 正则化强度参数

with pm.Model() as model_ridge:
    # Priors for unknown model parameters with regularization
    # a with ridge penalty (small sigma means regularization)
    a = pm.Normal('a', mu=0, sigma=1/np.sqrt(2*lambda_ridge))
    # b with ridge penalty
    b = pm.Normal('b', mu=0, sigma=1/np.sqrt(2*lambda_ridge))
    # c with ridge penalty
    c = pm.Normal('c', mu=0, sigma=1/np.sqrt(2*lambda_ridge))

    # Expected value of outcome
    mu = a * X ** 2 + b * X + c

    # Likelihood (sampling distribution) of observations
    Y = pm.Normal('Y', mu=mu, sigma=10, observed=Y_obs)

    # Sampling
    trace_ridge = pm.sample(1000, tune=1000, return_inferencedata=True)

# Plot the results
pm.plot_trace(trace_ridge)
plt.show()

In [ ]:
X_test = np.linspace(np.floor(X.min()), np.ceil(X.max()), 100)

# 真值
Y_true = 5 * X_test ** 2 + 10 * X_test

# 计算后验平均值
a_posterior_mean = trace_ridge.posterior["a"].mean().values
b_posterior_mean = trace_ridge.posterior["b"].mean().values
c_posterior_mean = trace_ridge.posterior["c"].mean().values
Y_fit = a_posterior_mean * X_test ** 2 + b_posterior_mean * X_test + c_posterior_mean

# 计算后验标准差
a_posterior_std = trace_ridge.posterior["a"].std().values
b_posterior_std = trace_ridge.posterior["b"].std().values
c_posterior_std = trace_ridge.posterior["c"].std().values
Y_errorpred = a_posterior_std * X_test ** 2 + b_posterior_std * X_test + c_posterior_std

edge1 = Y_fit + Y_errorpred
edge2 = Y_fit - Y_errorpred

crosspoint = 0
for i in range(len(X_test)):
    if edge1[i] < edge2[i]:
        crosspoint = i
        break

MSE = np.mean((Y_fit - Y_true) ** 2)

In [ ]:
# 绘制填充区间
# 绘制数据点
plt.scatter(X, Y_obs, s=5)

# 绘制后验平均拟合线
plt.plot(X_test, Y_fit, color='red', label='a={:.2f} b={:.2f} c={:.2f} MSE={:.2f}'.format(
         a_posterior_mean, b_posterior_mean, c_posterior_mean, MSE))

if crosspoint:
    plt.fill_between(X_test[:crosspoint], edge2[:crosspoint], edge1[:crosspoint], color='blue', alpha=0.3, 
                     label='Uncertainty')
    plt.fill_between(X_test[crosspoint:], edge1[crosspoint:], edge2[crosspoint:], color='blue', alpha=0.3)
else:
    plt.fill_between(X_test, edge2, edge1, color='blue', alpha=0.3)

# 显示图例
plt.legend()

# 显示图表
plt.show()